In [1]:
from Parameterize_and_testing_functions import *

import nglview as ng
import MDAnalysis as md
from MDAnalysis.analysis.leaflet import LeafletFinder, optimize_cutoff

# Import the 'module' function into python session
from lmod_env import module
module('load','GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2')

%matplotlib inline

insane = '/projects/cp/users/kmzh811/PYTHON/M3-Ionizable-Lipids/Collection_of_itps/insane-ILs.py'
mdp_loc_BL = '/projects/cp/user/kmzh811/MONOLAYERS/20nt_RNA_MC3_monolayers/MDPs'
IL_itp = '/projects/cp/users/kmzh811/PYTHON/M3-Ionizable-Lipids/Collection_of_itps/MC3_KC2_DP_DT_LI5.itp'
nt20RNA_itp = '/projects/cp/user/kmzh811/MONOLAYERS/20nt_RNA_MC3_monolayers/ssRNA_polyA.itp'
nt10RNA_tip = '/projects/cp/user/kmzh811/MONOLAYERS/10nt_RNA_MC3_monolayers/polyA_10.itp'
main_dir = '/projects/cp/user/kmzh811/MONOLAYERS/20nt_RNA_Lipid5_monolayers/' ### THIS ONE YOU HAVE TO CHANGE

In [2]:
RNA10nt_box = '/projects/cp/user/kmzh811/MONOLAYERS/10nt_RNA_MC3_monolayers/RNA_box.pdb' #34 molecules
RNA20nt_box = '/projects/cp/user/kmzh811/MONOLAYERS/20nt_RNA_MC3_monolayers/RNA_box.pdb' #34 molecules

RNA20nt_box_big = '/projects/cp/user/kmzh811/MONOLAYERS/20nt_RNA_MC3_monolayers/RNA_box_68.pdb' #68 molecules

RNA20nt_338  = '/projects/cp/user/kmzh811/MONOLAYERS/10nt_RNA_MC3_monolayers/RNA_box_338.pdb'

# Bilayer - Then separate into monolayers and possibly add RNA

In [3]:
def bilayer (insane_flag, IL1, IL2, RNA_itp='/projects/cp/user/kmzh811/MONOLAYERS/20nt_RNA_MC3_monolayers/ssRNA_polyA.itp'):
    insane = '/projects/cp/users/kmzh811/PYTHON/M3-Ionizable-Lipids/Collection_of_itps/insane-ILs.py'
    mdp_loc_BL = '/projects/cp/user/kmzh811/MONOLAYERS/20nt_RNA_MC3_monolayers/MDPs'
    IL_itp = '/projects/cp/users/kmzh811/PYTHON/M3-Ionizable-Lipids/Collection_of_itps/MC3_KC2_DP_DT_LI5.itp'
    nt20RNA_itp = '/projects/cp/user/kmzh811/MONOLAYERS/20nt_RNA_MC3_monolayers/ssRNA_polyA.itp'
    RNA_itp = '/projects/cp/user/kmzh811/MONOLAYERS/10nt_RNA_MC3_monolayers/polyA_10.itp'
    nt10RNA_tip = '/projects/cp/user/kmzh811/MONOLAYERS/10nt_RNA_MC3_monolayers/polyA_10.itp'
    os.system(f'python2 {insane} -dm 5 {insane_flag}  -pbc cubic -x 20 -y 20 -z 13 -o mem.gro -p topol.top -sol W') #x,y was 15 before

    prepare_setup_BL(RNA_itp, mdp_loc_BL, f'{IL1} {IL2}', 4, 0)

    u = md.Universe('m.gro')
    sel  = np.average(u.select_atoms(f'(resname {IL1} {IL2}) and not name N1 NP GL1 CN CX GLA GLB CA CB or resname CHOL and not name ROH').positions[:,2])

    x = u.atoms.positions[:,0]
    y = u.atoms.positions[:,1]
    z = np.zeros([x.shape[0]])

    z[:] = sel

    u.atoms.positions = np.vstack((x,y,z)).T

    u.atoms.write('REF.gro')

    ### Relax the system
    subprocess.call("gmx grompp -f {}/rel_310.mdp -c m.gro -r REF.gro -p topol.top -o r1.tpr -n index.ndx -maxwarn 4".format(mdp_loc_BL)
                    , shell = True)

    with open('submit_rel.sh','w') as su:
        su.write('#!/bin/bash\n')
        su.write(f'#SBATCH --job-name="{IL1}_RNA"\n')
        su.write('#SBATCH --output=job.out\n')
        su.write('#SBATCH --error=job.err\n')
        su.write('#SBATCH --time=02:00:00\n')
        su.write('#SBATCH -p core \n')
        su.write('#SBATCH -N 1\n')
        su.write('#SBATCH --ntasks-per-node=4\n')
        su.write('#SBATCH --cpus-per-task=1\n')
        su.write('#SBATCH --hint=nomultithread\n')

        su.write('\n')
        su.write('module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2\n')
        su.write('\n')
        su.write('export OMP_NUM_THREADS=4\n')
        su.write('\n')
        su.write('srun gmx_mpi mdrun -ntomp 4 -deffnm r1 -maxh 1.5\n')
    su.close()
    return

In [4]:
def generate_top_bot ():
    u = md.Universe('r1.gro')

    midt = u.select_atoms('name N1 NP PO4').center_of_geometry()[2]
    top = u.select_atoms(f'name N1 NP PO4 ROH and prop z > {midt}')
    bot = u.select_atoms(f'name N1 NP PO4 ROH and prop z < {midt}')
    mem_thick = (top.center_of_geometry() - bot.center_of_geometry())[2]

    RNA_lenght = 50
    trans = mem_thick * 2 + (mem_thick/2) + RNA_lenght 

    #mem_thickort moved bottom leaflet
    resids_bot = ' '.join(f'{i}' for i in bot.resids)
    bottom = u.select_atoms(f'resid {resids_bot}') 

    x = bottom.positions[:,0]
    y = bottom.positions[:,1]
    z = bottom.positions[:,2]

    z_new = z + (trans) 

    new_pos = np.vstack((x,y,z_new)).T

    bottom.positions = new_pos - bottom.center_of_geometry()

    bottom.atoms.write('bot.pdb')

    #Now export top leaflet
    resids_top = ' '.join(f'{i}' for i in top.resids)
    top_leaf = u.select_atoms(f'resid {resids_top}') 
    top_leaf.positions = top_leaf.positions - top_leaf.center_of_geometry()

    top_leaf.atoms.write('top.pdb')
    return trans, mem_thick

In [5]:
def combine_system2 (trans, RNA_box, ILs, no_RNA=34, add_rna=True):
    water_box = '/projects/cp/user/kmzh811/PYTHON/water.gro'
    if add_rna == True:
        u_rna = md.Universe(RNA_box)
        sel = u_rna.select_atoms('all')
        sel.positions = sel.positions - sel.center_of_geometry()
        u_rna.atoms.write('rna.gro')
        RNA = md.Universe('rna.gro')

    ## Join the two monolayers and make the box info in the gro file

    B = md.Universe('bot.pdb')
    T = md.Universe('top.pdb')

    sys = md.Merge(B.select_atoms('all'), T.select_atoms('all'))
    sys.atoms.write('stacked_monolayers.gro')

    xx = T.dimensions[0]/10
    yy = T.dimensions[1]/10
    zz = (T.dimensions[2]/10) + (trans/10) 

    stacked = md.Universe('stacked_monolayers.gro')
    sel = stacked.select_atoms('all')
    sel.positions = sel.positions - sel.center_of_geometry()
    if add_rna == True:
        sys_rna = md.Merge(stacked.select_atoms('all'), RNA.select_atoms('all'))
        sys_rna.atoms.write('stacked_monolayers_rna.gro')
        #### Renumber and generate box
        subprocess.call('gmx editconf -f stacked_monolayers_rna.gro -o stacked_monolayers_rna.gro -resnr 1 -box {} {} {} -c'.format(xx,yy,zz), shell=True)
        subprocess.call('gmx editconf -f stacked_monolayers_rna.gro -o stacked_monolayers_rna.pdb -resnr 1 -box {} {} {} -c'.format(xx,yy,zz), shell=True)
    else:
        #### Renumber and generate box
        subprocess.call('gmx editconf -f stacked_monolayers.gro -o stacked_monolayers.gro -resnr 1 -box {} {} {} -c'.format(xx,yy,zz), shell=True)
        subprocess.call('gmx editconf -f stacked_monolayers.gro -o stacked_monolayers.pdb -resnr 1 -box {} {} {} -c'.format(xx,yy,zz), shell=True)


    #### Solvate in the middle compartment with insane

    #solvate with insane
    if add_rna == True:
        u = md.Universe('stacked_monolayers_rna.gro')
    else:
        u = md.Universe('stacked_monolayers.gro')

    minn = np.min(u.select_atoms(f'resname {ILs}').positions[:,2])
    maxx = np.max(u.select_atoms(f'resname {ILs}').positions[:,2])

    diff = ((maxx-minn)/10) - 2
    
    if add_rna == True:
        os.system(f'gmx solvate -cp stacked_monolayers_rna.gro -cs {water_box}  -radius 0.21 -o mono_solv.gro')
    else:
        os.system(f'gmx solvate -cp stacked_monolayers.gro -cs {water_box}  -radius 0.21 -o mono_solv.gro')
    
            
    u = md.Universe('mono_solv.gro')
    L = LeafletFinder(u, 'name NP N1 ROH PO4')
    leaflet0 = L.groups(0)
    leaflet1 = L.groups(1)
    
    leaflets = [ leaflet0, leaflet1]
    zs = [leaflet0.center_of_mass()[2],leaflet1.center_of_mass()[2]]
    max_z = np.max(zs)
    idx = np.where(zs == max_z)
    min_z = np.min(zs)
    idm = np.where(zs == min_z)

    top_leaf = leaflets[[0][0]]
    bot_leaf = leaflets[idm[0][0]]
        
    sel = u.select_atoms(f'all and not (name W and prop z >= {top_leaf.center_of_mass()[2]}) and not (name W and prop z <= {bot_leaf.center_of_mass()[2]})')
    sel.atoms.write('mono_solv.pdb')
        
    subprocess.call('gmx editconf -f mono_solv.pdb -o mono_solv_box.gro -box {} {} {} -c'.format(xx,yy,zz), shell=True)

    u = md.Universe('mono_solv_box.gro')
    noW = u.select_atoms('name W').n_atoms

    #Update the topology file
    top_new = open('mono.top','w')
    top = open('topol.top', 'r').readlines()
    for line in top:

        if re.match('NA', line):
            continue
        elif re.match('CL', line):
            continue
        elif re.match('W', line):
            continue
        else:
            top_new.write(line)
    if add_rna == True:
        top_new.write(f'RNA   {no_RNA}\n')
    top_new.write(f'W    {noW}\n')
    top_new.close()

    lipids='POPC DPPC DIPC DPSM CHOL DBPC DOPC MC3 MC3H LI5H LI5'
    nt = 10
    pinoffset = 0 

    ### Neutralize
    subprocess.call('gmx grompp -f {}/min.mdp -c mono_solv_box.gro -p mono.top -o memion.tpr -r mono_solv_box.gro -maxwarn 3'.format(mdp_loc_BL), shell=True)    

    ### Create the index file
    gro = 'mono_solv_box.gro'
    u = md.Universe(gro)
    lipidsagg=u.select_atoms('resname {}'.format(lipids))
    solventagg=u.select_atoms('name W')
    lipidsagg.write("index.ndx", mode="w", name= 'Lipids')
    solventagg.write("index.ndx", mode="a", name= 'W')
    u.atoms.write("index.ndx", mode="a", name= 'System')

    with open('mono.top') as topinput:
        top = topinput.readlines()


    p = subprocess.Popen('gmx genion -s memion.tpr -o memion.gro -p mono.top -pname NA -pq +1 -nname CL -nq -1 -neutral -n index.ndx'
                            , stdin=subprocess.PIPE, shell=True, universal_newlines=True)
    p.communicate('W')
    p.wait()

    ### add 150mM NaCl
    Waterline =  top[-1].split(' ')
    Waternumber = int( Waterline[-1].rstrip() )
    naclNUM = int((0.15 * Waternumber*4)/55.5)
    subprocess.call('gmx grompp -f {}/min.mdp -c memion.gro -r memion.gro -p mono.top -o memion_2.tpr -maxwarn 3'.format(mdp_loc_BL)
                    , shell=True)

    p = subprocess.Popen('gmx genion -s memion_2.tpr -o memion_2.gro -p mono.top -pname NA -pq +1 -nname CL -nq -1 -np {} -nn {} -n index.ndx'.format(naclNUM, naclNUM)
                             , stdin=subprocess.PIPE, shell=True, universal_newlines=True)
    p.communicate('W')
    p.wait()

    ### Minimize the systemmdp
    subprocess.call("gmx grompp -f {}/min.mdp -c memion_2.gro -r memion_2.gro -p mono.top -o m.tpr -maxwarn 3".format(mdp_loc_BL)
                    , shell = True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
    os.system('gmx mdrun -v -deffnm m -nt 5')
#     subprocess.call("gmx mdrun -v -deffnm m -pin on "
#                     , shell = True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

    ### Create the index file
    gro = 'm.gro'
    u = md.Universe(gro)
    lipidsagg=u.select_atoms('resname {}'.format(lipids))
    solventagg=u.select_atoms('not resname {}'.format(lipids))
    lipidsagg.write("index.ndx", mode="w", name= 'Lipids')
    solventagg.write("index.ndx", mode="a", name= 'Solvent')
    u.atoms.write("index.ndx", mode="a", name= 'System')


def combine_system (trans, RNA_box, ILs, no_RNA=34, add_rna=True):
    if add_rna == True:
        u_rna = md.Universe(RNA_box)
        sel = u_rna.select_atoms('all')
        sel.positions = sel.positions - sel.center_of_geometry()
        rna = sel.translate([0,0,int(mem_thick*1.5)])
        u_rna.atoms.write('rna.gro')
        RNA = md.Universe('rna.gro')

    ## Join the two monolayers and make the box info in the gro file

    B = md.Universe('bot.pdb')
    T = md.Universe('top.pdb')

    sys = md.Merge(B.select_atoms('all'), T.select_atoms('all'))
    sys.atoms.write('stacked_monolayers.gro')

    xx = T.dimensions[0]/10
    yy = T.dimensions[1]/10
    #print ('#### X Coor####')
    #print (xx)
    #print ('#### Y Coor####')
    #print (yy)

    zz = (T.dimensions[2]/10) + (trans/10) 

    stacked = md.Universe('stacked_monolayers.gro')
    if add_rna == True:
        sys_rna = md.Merge(stacked.select_atoms('all'), RNA.select_atoms('all'))
        sys_rna.atoms.write('stacked_monolayers_rna.gro')
        #### Renumber and generate box
        subprocess.call('gmx editconf -f stacked_monolayers_rna.gro -o stacked_monolayers_rna.gro -resnr 1 -box {} {} {} -c'.format(xx,yy,zz), shell=True)
        subprocess.call('gmx editconf -f stacked_monolayers_rna.gro -o stacked_monolayers_rna.pdb -resnr 1 -box {} {} {} -c'.format(xx,yy,zz), shell=True)
    else:
        #### Renumber and generate box
        subprocess.call('gmx editconf -f stacked_monolayers.gro -o stacked_monolayers.gro -resnr 1 -box {} {} {} -c'.format(xx,yy,zz), shell=True)
        subprocess.call('gmx editconf -f stacked_monolayers.gro -o stacked_monolayers.pdb -resnr 1 -box {} {} {} -c'.format(xx,yy,zz), shell=True)


    #### Solvate in the middle compartment with insane

    #solvate with insane
    if add_rna == True:
        u = md.Universe('stacked_monolayers_rna.gro')
    else:
        u = md.Universe('stacked_monolayers.gro')

    minn = np.min(u.select_atoms(f'resname {ILs}').positions[:,2])
    maxx = np.max(u.select_atoms(f'resname {ILs}').positions[:,2])

    diff = ((maxx-minn)/10) - 2
    if add_rna == True:
        os.system(f'python2 {insane} -f stacked_monolayers_rna.gro  -x {xx} -y {yy} -z {diff} -pbc cubic -sol W -o mono_solv.gro -solr 1.3')
    else:
        os.system(f'python2 {insane} -f stacked_monolayers.gro  -x {xx} -y {yy} -z {diff} -pbc cubic -sol W -o mono_solv.gro')
    subprocess.call('gmx editconf -f mono_solv.gro -o mono_solv_box.gro -box {} {} {} -c'.format(xx,yy,zz), shell=True)

    u = md.Universe('mono_solv_box.gro')
    noW = u.select_atoms('name W').n_atoms

    #Update the topology file
    top_new = open('mono.top','w')
    top = open('topol.top', 'r').readlines()
    for line in top:

        if re.match('NA', line):
            continue
        elif re.match('CL', line):
            continue
        elif re.match('W', line):
            continue
        else:
            top_new.write(line)
    if add_rna == True:
        top_new.write(f'RNA   {no_RNA}\n')
    top_new.write(f'W    {noW}\n')
    top_new.close()

    lipids='POPC DPPC DIPC DPSM CHOL DBPC DOPC MC3 MC3H LI5H LI5'
    nt = 10
    pinoffset = 0 

    ### Neutralize
    subprocess.call('gmx grompp -f {}/min.mdp -c mono_solv_box.gro -p mono.top -o memion.tpr -r mono_solv_box.gro -maxwarn 3'.format(mdp_loc_BL), shell=True)    

    ### Create the index file
    gro = 'mono_solv_box.gro'
    u = md.Universe(gro)
    lipidsagg=u.select_atoms('resname {}'.format(lipids))
    solventagg=u.select_atoms('name W')
    lipidsagg.write("index.ndx", mode="w", name= 'Lipids')
    solventagg.write("index.ndx", mode="a", name= 'W')
    u.atoms.write("index.ndx", mode="a", name= 'System')

    with open('mono.top') as topinput:
        top = topinput.readlines()


    p = subprocess.Popen('gmx genion -s memion.tpr -o memion.gro -p mono.top -pname NA -pq +1 -nname CL -nq -1 -neutral -n index.ndx'
                            , stdin=subprocess.PIPE, shell=True, universal_newlines=True)
    p.communicate('W')
    p.wait()

    ### add 150mM NaCl
    Waterline =  top[-1].split(' ')
    Waternumber = int( Waterline[-1].rstrip() )
    naclNUM = int((0.15 * Waternumber*4)/55.5)
    subprocess.call('gmx grompp -f {}/min.mdp -c memion.gro -r memion.gro -p mono.top -o memion_2.tpr -maxwarn 3'.format(mdp_loc_BL)
                    , shell=True)

    p = subprocess.Popen('gmx genion -s memion_2.tpr -o memion_2.gro -p mono.top -pname NA -pq +1 -nname CL -nq -1 -np {} -nn {} -n index.ndx'.format(naclNUM, naclNUM)
                             , stdin=subprocess.PIPE, shell=True, universal_newlines=True)
    p.communicate('W')
    p.wait()

    ### Minimize the systemmdp
    subprocess.call("gmx grompp -f {}/min.mdp -c memion_2.gro -r memion_2.gro -p mono.top -o m.tpr -maxwarn 3".format(mdp_loc_BL)
                    , shell = True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
    subprocess.call("gmx mdrun -v -deffnm m -nt {0} -pin on -pinstride 1 -pinoffset {1}".format(nt, pinoffset)
                    , shell = True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

    ### Create the index file
    gro = 'm.gro'
    u = md.Universe(gro)
    lipidsagg=u.select_atoms('resname {}'.format(lipids))
    solventagg=u.select_atoms('not resname {}'.format(lipids))
    lipidsagg.write("index.ndx", mode="w", name= 'Lipids')
    solventagg.write("index.ndx", mode="a", name= 'Solvent')
    u.atoms.write("index.ndx", mode="a", name= 'System')


In [6]:
def rel(mdp_loc_BL='/projects/cp/user/kmzh811/MONOLAYERS/20nt_RNA_MC3_monolayers/MDPs'):
    #Make a reference file for the restraints 
    u = md.Universe('m.gro')
    sel  = np.average(u.select_atoms('all').positions[:,2])

    x = u.atoms.positions[:,0]
    y = u.atoms.positions[:,1]
    z = np.zeros([x.shape[0]])

    z[:] = sel

    u.atoms.positions = np.vstack((x,y,z)).T

    u.atoms.write('REF.gro')

    
    subprocess.call("gmx grompp -f {}/rel_310_XY_constant.mdp -c m.gro -r m.gro -p mono.top -o r.tpr -maxwarn 4 -n index.ndx".format(mdp_loc_BL)
                    , shell = True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
    with open('submit_rel.sh','w') as su:
        su.write('#!/bin/bash\n')
        su.write('#SBATCH --job-name="MONO_REL"\n')
        su.write('#SBATCH --output=job.out\n')
        su.write('#SBATCH --error=job.err\n')
        su.write('#SBATCH --time=02:00:00\n')
        su.write('#SBATCH -p core \n')
        #su.write('#SBATCH --gres=gpu:1\n')
        su.write('#SBATCH -N 1\n')
        su.write('#SBATCH --ntasks-per-node=1\n')
        su.write('#SBATCH --cpus-per-task=2\n')
        su.write('#SBATCH --hint=nomultithread\n')

        su.write('\n')
        su.write('module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2\n')
        su.write('\n')
        su.write('export OMP_NUM_THREADS=2\n')
        su.write('\n')
        su.write('srun gmx_mpi mdrun -ntomp 2 -deffnm r -maxh 1.5\n')
    su.close()
    return

def prod (mdp_loc_BL='/projects/cp/user/kmzh811/MONOLAYERS/20nt_RNA_MC3_monolayers/MDPs'):
    #production run gromp
    subprocess.call("gmx grompp -f {}/prod_310_XY_constant.mdp -c r.gro -r mem.gro -p mono.top -o p.tpr -maxwarn 3 -n index.ndx".format(mdp_loc_BL)
                    , shell = True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
    with open('submit_pro.sh','w') as su:
        su.write('#!/bin/bash\n')
        su.write('#SBATCH --job-name="MONO_PROD"\n')
        su.write('#SBATCH --output=job.out\n')
        su.write('#SBATCH --error=job.err\n')
        su.write('#SBATCH --time=48:0:0\n')
        su.write('#SBATCH -p gpu \n')
        su.write('#SBATCH --gres=gpu:1\n')
        su.write('#SBATCH -N 1\n')
        su.write('#SBATCH --ntasks-per-node=1\n')
        su.write('#SBATCH --cpus-per-task=10\n')
        su.write('#SBATCH --hint=nomultithread\n')

        su.write('\n')
        su.write('module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2\n')
        su.write('\n')
        su.write('export OMP_NUM_THREADS=10\n')
        su.write('\n')
        su.write('srun gmx_mpi mdrun -ntomp 10 -deffnm p -maxh 47.5\n')
    su.close()
    return


def rel_prod (mdp_loc_BL='/projects/cp/user/kmzh811/MONOLAYERS/20nt_RNA_MC3_monolayers/MDPs'):
    with open('submit_pro.sh','w') as su:
        su.write('#!/bin/bash\n')
        su.write('#SBATCH --job-name="MONO_PROD"\n')
        su.write('#SBATCH --output=job.out\n')
        su.write('#SBATCH --error=job.err\n')
        su.write('#SBATCH --time=100:0:0\n')
        su.write('#SBATCH -p gpu \n')
        su.write('#SBATCH --gres=gpu:1\n')
        su.write('#SBATCH -N 1\n')
        su.write('#SBATCH --ntasks-per-node=2\n')
        su.write('#SBATCH --cpus-per-task=10\n')
        su.write('#SBATCH --hint=nomultithread\n')

        su.write('\n')
        su.write('module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2\n')
        su.write('\n')
        su.write('export OMP_NUM_THREADS=10\n')
        su.write('\n')
        su.write(f'gmx grompp -f {mdp_loc_BL}/rel_310_NVT.mdp -c m.gro -r m.gro -p mono.top -o r.tpr -maxwarn 4 -n index.ndx\n')
        su.write('srun gmx_mpi mdrun -ntomp 10 -deffnm r -maxh 47.5\n')
        su.write(f'gmx grompp -f {mdp_loc_BL}/prod_310_NVT.mdp -c r.gro -r m.gro -p mono.top -o p.tpr -maxwarn 4 -n index.ndx\n')
        su.write('srun gmx_mpi mdrun -ntomp 10 -deffnm p -maxh 47.5\n')
    su.close()
    return

In [ ]:
LI5H_range = [100, 75, 57]
CHOL_range = [0, 25, 43]
ILs = 'LI5 LI5H'

for idx, m in enumerate(LI5H_range):
    if CHOL_range[idx] != 0 :
        insane_flag = f'-l LI5H:{m} -l CHOL:{CHOL_range[idx]}'
    else:
        insane_flag = f'-l LI5H:{m}'
    dir_name = f'LI5H_CHOL_{m}_{CHOL_range[idx]}_RNA'
    
    try:
        os.chdir(f'{main_dir}/{dir_name}')
    except:
        os.mkdir(f'{main_dir}/{dir_name}')
        os.chdir(f'{main_dir}/{dir_name}')
    ## Bilayer first
    bilayer(insane_flag, 'LI5H', '', RNA_itp=nt10RNA_tip) #in here set the correct RNA itp
    os.system('gmx mdrun -v -deffnm r1 -ntomp 2 -ntmpi 2 -pin on')
    
    ## Then translate monlayers
    trans, mem_thick = generate_top_bot()
    
    ## Combine into one system and solvate 
    combine_system2 (trans, RNA10nt_box, ILs, no_RNA=0, add_rna=False)
    
    ## Relaxtation and productionm
    os.system(f'cp /projects/cp/user/kmzh811/MONOLAYERS/SUBMIT/*sh .')
    os.system('gmx grompp -f /projects/cp/user/kmzh811/MONOLAYERS/20nt_RNA_MC3_monolayers/MDPs/rel_310_NVT.mdp -c m.gro -r m.gro -p mono.top -o r.tpr -maxwarn 4 -n index.ndx')
    os.system('gmx mdrun -v -deffnm r1 -ntomp 2 -ntmpi 2 -pin on')
    for i in range(1,4):
        os.system(f'gmx grompp -f /projects/cp/user/kmzh811/MONOLAYERS/20nt_RNA_MC3_monolayers/MDPs/prod_310_NVT.mdp -c r.gro -r m.gro -p mono.top -o p_{i}.tpr -maxwarn 4 -n index.ndx')

; X: 20.000 (26 bins) Y: 20.000 (26 bins) in upper leaflet
; X: 20.000 (26 bins) Y: 20.000 (26 bins) in lower leaflet
; 676 lipids in upper leaflet, 676 lipids in lower leaflet
; NDX Solute 1 0
; Charge of protein: 0.000000
; NDX Membrane 1 17576
; Charge of membrane: 0.000000
; Total charge: 0.000000
; NDX Solvent 17577 45076
; NDX System 1 45076
; "I mean, the good stuff is just INSANE" --Julia Ormond
                  :-) GROMACS - gmx grompp, 2021-MODIFIED (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric I

Setting the LD random seed to -1615861858

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'LI5H'

Excluding 1 bonded neighbours molecule type 'LI5H'

Excluding 1 bonded neighbours molecule type 'W'
Analysing residue names:
There are: 28852      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

This run will generate roughly 5 Mb of data


                  :-) GROMACS - gmx genion, 2021-MODIFIED (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu

Will try to add 0 NA ions and 1352 CL ions.
Select a continuous group of solvent molecules
Selected 1: 'W'

Processing topology
Replacing 1352 solute molecules in topology file (topol.top)  by 0 NA and 1352 CL ions.


                  :-) GROMACS - gmx genion, 2021-MODIFIED (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu

...

                  :-) GROMACS - gmx genion, 2021-MODIFIED (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu

Will try to add 474 NA ions and 474 CL ions.
Select a continuous group of solvent molecules
Selected 1: 'W'

Processing topology
Replacing 948 solute molecules in topology file (mono.top)  by 474 NA and 474 CL ions.


                   :-) GROMACS - gmx mdrun, 2021-MODIFIED (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu

Setting the LD random seed to -272629827

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'LI5H'

Excluding 1 bonded neighbours molecule type 'LI5H'

Excluding 1 bonded neighbours molecule type 'W'

Excluding 1 bonded neighbours molecule type 'NA'

Excluding 1 bonded neighbours molecule type 'CL'

Excluding 1 bonded neighbours molecule type 'CL'

Setting gen_seed to 2128606109

Velocities were taken from a Maxwell distribution at 310 K

The center of mass of the position restraint coord's is  0.094  3.468  7.095

The center of mass of the position restraint coord's is  0.094  3.468  7.095

This run will generate roughly 299 Mb of data



GROMACS reminds you: "Time is the best appraiser of scientific work, and I am aware that an industrial discovery rarely produces all its fruit in the hands of its first inventor." (Louis Pasteur)

                   :-) GROMACS - gmx mdrun, 2021-MODIFIED (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner           

In [ ]:
LI5_range = [100, 75, 57]
CHOL_range = [0, 25, 43]
ILs = 'LI5 LI5H'

for idx, m in enumerate(LI5_range):
    if CHOL_range[idx] != 0 :
        insane_flag = f'-l LI5:{m} -l CHOL:{CHOL_range[idx]}'
    else:
        insane_flag = f'-l LI5:{m}'
    dir_name = f'LI5_CHOL_{m}_{CHOL_range[idx]}_RNA'
    
    try:
        os.chdir(f'{main_dir}/{dir_name}')
    except:
        os.mkdir(f'{main_dir}/{dir_name}')
        os.chdir(f'{main_dir}/{dir_name}')
    ## Bilayer first
    bilayer(insane_flag, 'LI5', '', RNA_itp=nt10RNA_tip) #in here set the correct RNA itp
    os.system('gmx mdrun -v -deffnm r1 -ntomp 2 -ntmpi 2 -pin on')
    
    ## Then translate monlayers
    trans, mem_thick = generate_top_bot()
    
    ## Combine into one system and solvate 
    combine_system2 (trans, RNA10nt_box, ILs, no_RNA=0, add_rna=False)
    
    ## Relaxtation and productionm
    os.system(f'cp /projects/cp/user/kmzh811/MONOLAYERS/SUBMIT/*sh .')
    os.system('gmx grompp -f /projects/cp/user/kmzh811/MONOLAYERS/20nt_RNA_MC3_monolayers/MDPs/rel_310_NVT.mdp -c m.gro -r m.gro -p mono.top -o r.tpr -maxwarn 4 -n index.ndx')
    os.system('gmx mdrun -v -deffnm r1 -ntomp 2 -ntmpi 2 -pin on')    for i in range(1,4):
    os.system(f'gmx grompp -f /projects/cp/user/kmzh811/MONOLAYERS/20nt_RNA_MC3_monolayers/MDPs/prod_310_NVT.mdp -c r.gro -r m.gro -p mono.top -o p_{i}.tpr -maxwarn 4 -n index.ndx')